In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import numpy as np


In [2]:
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')
order_products = pd.read_csv('order_products_s.csv')
orders = pd.read_csv('orders_s.csv')
products = pd.read_csv('products.csv')
tips = pd.read_csv('tips.csv')



In [3]:
tips.drop(columns="Unnamed: 0",inplace=True)

In [4]:
orders

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0
...,...,...,...,...,...,...
1673016,2266710,206209,10,5,18,29.0
1673017,1854736,206209,11,4,10,30.0
1673018,626363,206209,12,1,12,18.0
1673019,2977660,206209,13,1,12,7.0


In [5]:
# Verknüpfe Tipps mit Bestellungen
orders = orders.merge(tips, on='order_id', how='left')
#orders.dropna(inplace=True)  

In [6]:
orders

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip
0,2539329,1,1,2,8,NaN,False
1,2398795,1,2,3,7,15.0,False
2,473747,1,3,3,12,21.0,False
3,2254736,1,4,4,7,29.0,False
4,431534,1,5,4,15,28.0,False
...,...,...,...,...,...,...,...
1673016,2266710,206209,10,5,18,29.0,False
1673017,1854736,206209,11,4,10,30.0,False
1673018,626363,206209,12,1,12,18.0,False
1673019,2977660,206209,13,1,12,7.0,False


In [7]:
orders= orders.sort_values(by=["user_id","order_number"])

# Spalte für vorherige und vor-vorherige Trinkgeldangaben
orders['tip_prev'] = orders.groupby('user_id')['tip'].shift(1)
orders['tip_prev2'] = orders.groupby('user_id')['tip'].shift(2)


In [8]:
orders[orders["user_id"] == 67]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,tip_prev,tip_prev2
550,3205960,67,1,3,11,NaN,True,NaN,NaN
551,970922,67,2,1,15,5.0,True,True,NaN
552,596585,67,3,1,13,7.0,False,True,True
553,2037087,67,4,2,9,8.0,True,False,True
554,2474111,67,5,3,14,8.0,True,True,False
555,1568481,67,6,3,12,7.0,False,True,True
556,1723781,67,7,2,13,6.0,True,False,True
557,2048527,67,8,2,11,7.0,False,True,False
558,2389031,67,9,2,14,7.0,False,False,True
559,2378562,67,10,1,10,6.0,True,False,False


In [9]:
pd.crosstab(orders['tip_prev'], orders['tip'], margins=True,normalize='all')

tip,False,True,All
tip_prev,,,
False,0.432685,0.124802,0.557487
True,0.126103,0.316409,0.442513
All,0.558789,0.441211,1.000000


Ca. 44% aller Bestellungen erhalten Trinkgeld

In [16]:
pd.crosstab(orders['tip'], orders['tip_prev'], margins=True,normalize='index')

tip_prev,False,True
tip,,
False,0.774327,0.225673
True,0.282863,0.717137
All,0.557487,0.442513


Wenn ein Kunde bei der vorhergenden Bestellung Trinkgeld gibt, ist die Wahrscheinlichkeit bei der aktuellen Bestellung 71%, d.h. ein Zusammenhang besteht.

In [19]:
pd.crosstab(orders['tip'], orders['tip_prev2'], margins=True,normalize='index')

tip_prev2,False,True
tip,,
False,0.739978,0.260022
True,0.316907,0.683093
All,0.550662,0.449338


Ähnliche Zahlen, für die vor-vorgehende Bestellung. Besteht diese Zusammenhang durch tip_prev?

In [12]:


# b) Häufigkeiten und Wahrscheinlichkeiten basierend auf `tip_prev2`







tip_prev_counts.plot(kind='bar', stacked=True, figsize=(8, 6), color=['lightblue', 'orange'])
plt.title('Trinkgeldwahrscheinlichkeit basierend auf der vorherigen Bestellung')
plt.xlabel('Trinkgeld bei vorheriger Bestellung (True/False)')
plt.ylabel('Prozentualer Anteil')
plt.xticks(rotation=0)
plt.legend(title='Tip bei aktueller Bestellung')
plt.show()

# b) Trinkgeldwahrscheinlichkeit nach `tip_prev2`
tip_prev2_counts.plot(kind='bar', stacked=True, figsize=(8, 6), color=['lightgreen', 'purple'])
plt.title('Trinkgeldwahrscheinlichkeit basierend auf der vor-vorhergehenden Bestellung')
plt.xlabel('Trinkgeld bei vor-vorhergehender Bestellung (True/False)')
plt.ylabel('Prozentualer Anteil')
plt.xticks(rotation=0)
plt.legend(title='Tip bei aktueller Bestellung')
plt.show()



NameError: name 'tip_prev_counts' is not defined

In [17]:
pd.crosstab(orders['tip_prev2'], orders['tip_prev'], margins=True,normalize='index')

tip_prev,False,True
tip_prev2,,
False,0.770967,0.229033
True,0.283463,0.716537
All,0.551913,0.448087


In [18]:
orders.pivot_table('tip', 'tip_prev', 'tip_prev2', aggfunc=np.mean)

C:\Users\Benjamin\AppData\Local\Temp\ipykernel_14244\485001432.py:1: FutureWarning: The provided callable <function mean at 0x000002B23E91A020> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  orders.pivot_table('tip', 'tip_prev', 'tip_prev2', aggfunc=np.mean)


tip_prev2,False,True
tip_prev,,
False,0.174175,0.405815
True,0.538099,0.788844


Die Höchste Trinkgeldwahrscheinlichkeit für den Fall tip_prev=True und tip_prev2, D.h. tip_prev2 bietet uns unabhängig von trip_prev informationen über das Trinkgeldverhalten.